In [131]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [132]:
csitrade=pd.read_csv('E:/baselines/ZZ500.csv')
# csitrade.rename(columns={'Close':'close'},inplace=True)

In [133]:
#数据一定要按股票排序
csitrade['close_r']=csitrade['close']/csitrade['close'].shift(1)
csitrade.reset_index(inplace=True,drop=True)
csitrade['dt'] = csitrade['dt'].astype('datetime64[ns]')

In [134]:
csitrade

,kdcode,dt,close,open,high,low,turnover,volume,close_r
0,000009.SZ,2018-01-02,7.32,7.27,7.33,7.23,140497530.0,19269993.0,NaN
1,000009.SZ,2018-01-03,7.42,7.32,7.44,7.30,170636551.0,23092097.0,1.013661
2,000009.SZ,2018-01-04,7.64,7.52,7.70,7.51,358089946.0,46992707.0,1.029650
3,000009.SZ,2018-01-05,7.55,7.65,7.65,7.51,196259451.0,25990264.0,0.988220
4,000009.SZ,2018-01-08,7.52,7.55,7.63,7.46,178511079.0,23703431.0,0.996026
...,...,...,...,...,...,...,...,...,...
169007,603883.SH,2023-12-25,29.37,29.02,29.58,28.80,101007153.0,3457490.0,1.012061
169008,603883.SH,2023-12-26,29.05,29.26,29.49,28.82,66158144.0,2272587.0,0.989105
169009,603883.SH,2023-12-27,30.08,28.94,30.40,28.80,206474442.0,6939076.0,1.035456
169010,603883.SH,2023-12-28,28.98,30.15,30.20,28.44,264520627.0,9053474.0,0.963431


In [135]:
# a2c ddpg ppo td3 sac
drl = 'sac'
dtrader=pd.read_csv(f'DRL/{drl}/ZZ500.csv')
df_return=pd.DataFrame(dtrader[drl])
df_return.rename(columns={drl:'daily_return'}, inplace=True)
df_return=df_return/df_return.shift(1)
df_return.fillna(1,inplace=True)
df_return['daily_return']=df_return['daily_return']-1

In [136]:
#大盘和日收益率
import pandas as pd
df_trade=csitrade
df_trade.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
df_trade=df_trade[df_trade['datetime']>'2023-01-01']
df_trade.rename(columns={'close_r' : 'daily_return'}, inplace=True)
df_trade['daily_return'] = df_trade['daily_return'] - 1
index_df = df_trade.groupby('datetime')['daily_return'].mean().reset_index()

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_4184\774818839.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade.rename(columns={'close_r' : 'daily_return'}, inplace=True)
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_4184\774818839.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade['daily_return'] = df_trade['daily_return'] - 1


In [137]:
portfolio_df_performance = df_return
index_df_performance = index_df.set_index(['datetime'])

In [138]:
portfolio_df_performance

,daily_return
0,0.000000
1,-0.000809
2,0.000809
3,0.000623
4,0.004108
...,...
237,0.002970
238,-0.008770
239,0.008303
240,0.013748


In [139]:
ARR = (portfolio_df_performance['daily_return']+1).cumprod()
ARR.to_csv(f"DRL/daily_return/zz500/{drl}.csv", index=None)

In [140]:
ARR

0      1.000000
1      0.999191
2      0.999999
3      1.000622
4      1.004733
         ...   
237    0.819849
238    0.812659
239    0.819406
240    0.830671
241    0.839303
Name: daily_return, Length: 242, dtype: float64

In [141]:
index_df_performance

,daily_return
datetime,
2023-01-03,0.018333
2023-01-04,-0.000141
2023-01-05,0.006995
2023-01-06,-0.001129
2023-01-09,0.006703
...,...
2023-12-25,0.001412
2023-12-26,-0.010048
2023-12-27,0.006425


In [142]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['index_daily_return'] = index_df_performance['daily_return']
alpha_df_performance['alpha_daily_return'] = alpha_df_performance['portfolio_daily_return'] - \
                                                alpha_df_performance['index_daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()
alpha_df_performance['index_net_value'] = (alpha_df_performance['index_daily_return'] + 1).cumprod()
alpha_df_performance['alpha_net_value'] = (alpha_df_performance['alpha_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value', 'index_net_value', 'alpha_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old beha

In [143]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,-0.166751,0.081048,-0.206625,-2.057444,-0.807022,-2.2187,-0.160697,0.005106
index_net_value,-0.166751,NaN,-0.206625,NaN,-0.807022,NaN,-0.160697,NaN
alpha_net_value,-0.166751,NaN,-0.206625,NaN,-0.807022,NaN,-0.160697,NaN


ASR = ARR / AVol;   CR = ARR / abs(MDD)
              ARR    AVol    MDD    ASR     CR    IR
nas100 a2c   0.109  0.134  -0.114  0.816  0.957  0.844
       ddpg  0.130  0.156  -0.131  0.832  0.994  0.863
       ppo   0.148  0.118  -0.104  1.259  1.424  1.237
       td3   0.181  0.155  -0.160  1.169  1.130  1.156
       sac   0.162  0.139  -0.107  1.165  1.518  1.154
      
sp500  a2c   0.160  0.126  -0.084  1.267  1.907  1.244
       ddpg  0.111  0.129  -0.091  0.864  1.223  0.887
       ppo   0.020  0.089  -0.067  0.220  0.291  0.263
       td3   0.024  0.113  -0.105  0.209  0.225  0.264
       sac   0.140  0.111  -0.069  1.263  2.011  1.242
      
zz500  a2c  -0.172  0.084  -0.208 -2.043 -0.826 -2.207
       ddpg -0.128  0.082  -0.170 -1.563 -0.756 -1.639
       ppo  -0.032  0.015  -0.040 -2.041 -0.787 -2.075
       td3  -0.123  0.135  -0.248 -0.912 -0.496 -0.909
       sac  -0.167  0.081  -0.207 -2.057 -0.807 -2.219
      
hs300  a2c  -0.207  0.092  -0.259 -2.255 -0.803 -2.490
       ddpg -0.137  0.138  -0.240 -0.992 -0.568 -1.002
       ppo  -0.096  0.045  -0.120 -2.138 -0.800 -2.234
       td3  -0.154  0.137  -0.252 -1.122 -0.610 -1.155
       sac  -0.140  0.090  -0.207 -1.554 -0.676 -1.635
       